In [1]:
import os
import requests
import pandas as pd
import sqlite3

In [2]:
# diretorio destino
data_directory = "data"

In [7]:
# Baixar arquivos

# URL base dos datasets do IMDB
base_url = "https://datasets.imdbws.com/"

# arquivos para serem baixados
files = [
    "name.basics.tsv.gz",
    "title.akas.tsv.gz",
    "title.basics.tsv.gz",
    "title.crew.tsv.gz",
    "title.episode.tsv.gz",
    "title.principals.tsv.gz",
    "title.ratings.tsv.gz",
]

os.makedirs(data_directory, exist_ok=True)

# baixar os arquivos
for file in files:
    url = base_url + file
    destiny_file_name = os.path.join(data_directory, file)

    # verificação se o arquivo ja foi baixado
    if not os.path.exists(destiny_file_name):
        print(f'Baixando {file}...')
        response = requests.get(url)

        # verificação da resposta da página (200 para sucesso)
        if response.status_code == 200:
            with open(destiny_file_name, 'wb') as f:
                f.write(response.content)
            print(f'{file} baixado com sucesso!')    
        else:
            print(f'Falha ao baixar o {file}. Código de status {response.status_code}')
    else:
        print(f'{file} já existe. Download já foi executado antes')
print('Download concluído.')

name.basics.tsv.gz já existe. Download já foi executado antes
title.akas.tsv.gz já existe. Download já foi executado antes
title.basics.tsv.gz já existe. Download já foi executado antes
title.crew.tsv.gz já existe. Download já foi executado antes
title.episode.tsv.gz já existe. Download já foi executado antes
title.principals.tsv.gz já existe. Download já foi executado antes
title.ratings.tsv.gz já existe. Download já foi executado antes
Download concluído.


In [10]:
# tratamento de dados

# diretorio de dados tratados
processed_data_directory = os.path.join(data_directory, 'tratado')

# certificação de existência do diretorio 'tratado'
os.makedirs(processed_data_directory, exist_ok=True)

# lista dos arquivos de dataset compactados
files = os.listdir(data_directory)

# loop para abrir, tratar e salvar cada arquivo
for file in files:
    file_path = os.path.join(data_directory, file)

    if os.path.isfile(file_path) and file.endswith('.gz'):
        print(f'Lendo e tratando o arquivo {file}')
        # leitura do arquivo TSV usando o pandas
        df = pd.read_csv(file_path, sep='\t', compression='gzip', low_memory=False)
    
        # substitui os caracters '\N' por fvalor nulo
        df.replace({'\\N':None}, inplace=True)
    
        # salva o dataframe no diretório de tratado após a descompressão dos arquivos
        destiny_directory = os.path.join(processed_data_directory, file[:-3]) # remove a extensão .gz
        df.to_csv(destiny_directory, sep='\t', index=False)
    
        print(f'Tratamento concluído para {file}. Arquivo tratado salvo em {destiny_directory}')

print('Todos os arquivos foram tratados e salvos.')

Lendo e tratando o arquivo name.basics.tsv.gz
Tratamento concluído para name.basics.tsv.gz. Arquivo tratado salvo em data\tratado\name.basics.tsv
Lendo e tratando o arquivo title.akas.tsv.gz
Tratamento concluído para title.akas.tsv.gz. Arquivo tratado salvo em data\tratado\title.akas.tsv
Lendo e tratando o arquivo title.basics.tsv.gz
Tratamento concluído para title.basics.tsv.gz. Arquivo tratado salvo em data\tratado\title.basics.tsv
Lendo e tratando o arquivo title.crew.tsv.gz
Tratamento concluído para title.crew.tsv.gz. Arquivo tratado salvo em data\tratado\title.crew.tsv
Lendo e tratando o arquivo title.episode.tsv.gz
Tratamento concluído para title.episode.tsv.gz. Arquivo tratado salvo em data\tratado\title.episode.tsv
Lendo e tratando o arquivo title.principals.tsv.gz
Tratamento concluído para title.principals.tsv.gz. Arquivo tratado salvo em data\tratado\title.principals.tsv
Lendo e tratando o arquivo title.ratings.tsv.gz
Tratamento concluído para title.ratings.tsv.gz. Arquivo tr

In [3]:
# validando dataframe
df = pd.read_csv('./data/tratado/name.basics.tsv', sep='\t')
df.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0027125,tt0031983"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0038355,tt0117057"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0050986,tt0069467,tt0050976,tt0083922"


In [14]:
# verificação de dados nulos por coluna
df.isnull().sum()

nconst                      0
primaryName                60
birthYear            13626211
deathYear            14026006
primaryProfession     2795186
knownForTitles        1629078
dtype: int64

In [2]:
data_base = 'imdb_data.db'

In [16]:
# Salvando em banco de dados com SQLite

# diretorios
processed_data_directory = os.path.join('data', 'tratado')

# conecta ao banco os dados SQLite
connection = sqlite3.connect(data_base)

# lista todos os arquivos do diretorio 'tratado'
files = os.listdir(processed_data_directory)

# loop para ler cada arquivo e salvar em tabela sqlite
for file in files:
    dir_file = os.path.join(processed_data_directory, file)

    if os.path.isfile(dir_file) and file.endswith('.tsv'):
        # lê o arquivo tsv com o pandas
        df = pd.read_csv(dir_file, sep='\t', low_memory=False)

        # remove a extensão do nome do arquivo
        table_name = os.path.splitext(file)[0]

        # substitui caracteres '.' e '-' para '_'
        table_name = table_name.replace('.', '_').replace('-', '_')

        # salva o dataframe na tabela sqlite
        df.to_sql(table_name, connection, index=False, if_exists='replace')

        print(f'Arquivo {file} salvo como tabela {table_name} no banco de dados')

# fecha a conexão com o banco de dados
connection.close()

print('Todos os arquivos foram salvos no banco de dados')

Arquivo name.basics.tsv salvo como tabela name_basics no banco de dados
Arquivo title.akas.tsv salvo como tabela title_akas no banco de dados
Arquivo title.basics.tsv salvo como tabela title_basics no banco de dados
Arquivo title.crew.tsv salvo como tabela title_crew no banco de dados
Arquivo title.episode.tsv salvo como tabela title_episode no banco de dados
Arquivo title.principals.tsv salvo como tabela title_principals no banco de dados
Arquivo title.ratings.tsv salvo como tabela title_ratings no banco de dados
Todos os arquivos foram salvos no banco de dados


In [3]:
# conexão com o banco
connection = sqlite3.connect(data_base)

# cria um cursor
cursor = connection.cursor()

# query SQL para obter o nome das tabelas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# recupera os resultados da consulta
tables = cursor.fetchall()

# Exibe o nome das tabelas
print('Tabelas do banco de dados:')
for table in tables:
    print(table[0])

# fecha o cursor e a conexão com o banco de dados
cursor.close()
connection.close()

Tabelas do banco de dados:
name_basics
title_akas
title_basics
title_crew
title_episode
title_principals
title_ratings
analitico_titulos
analitico_participantes


In [4]:
# conexão ao banco
connection = sqlite3.connect(data_base)

# query para obter as 10 primeiras linhas da tabela name_basics
query = 'SELECT * FROM name_basics LIMIT 10'
df = pd.read_sql_query(query, connection)

# fecha a conexão
connection.close()
df

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0027125,tt0031983"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0038355,tt0117057"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0050986,tt0069467,tt0050976,tt0083922"
5,nm0000006,Ingrid Bergman,1915.0,1982.0,"actress,producer,soundtrack","tt0034583,tt0038109,tt0036855,tt0038787"
6,nm0000007,Humphrey Bogart,1899.0,1957.0,"actor,producer,miscellaneous","tt0034583,tt0043265,tt0037382,tt0033870"
7,nm0000008,Marlon Brando,1924.0,2004.0,"actor,director,writer","tt0078788,tt0068646,tt0047296,tt0044081"
8,nm0000009,Richard Burton,1925.0,1984.0,"actor,producer,director","tt0061184,tt0087803,tt0059749,tt0057877"
9,nm0000010,James Cagney,1899.0,1986.0,"actor,director,producer","tt0029870,tt0031867,tt0042041,tt0034236"


In [6]:
connection = sqlite3.connect(data_base)

query = """
SELECT 
    tp.nconst,
    tp.tconst,
    tp.ordering,
    tp.category,
    tb.genres
FROM title_principals as tp
LEFT JOIN title_basics as tb
    ON tb.tconst = tp.tconst

LIMIT 10
"""
df = pd.read_sql_query(query, connection)

connection.close()

In [7]:
df

,nconst,tconst,ordering,category,genres
0,nm1588970,tt0000001,1,self,"Documentary,Short"
1,nm0005690,tt0000001,2,director,"Documentary,Short"
2,nm0005690,tt0000001,3,producer,"Documentary,Short"
3,nm0374658,tt0000001,4,cinematographer,"Documentary,Short"
4,nm0721526,tt0000002,1,director,"Animation,Short"
5,nm1335271,tt0000002,2,composer,"Animation,Short"
6,nm0721526,tt0000003,1,director,"Animation,Comedy,Romance"
7,nm0721526,tt0000003,2,writer,"Animation,Comedy,Romance"
8,nm1770680,tt0000003,3,producer,"Animation,Comedy,Romance"
9,nm0721526,tt0000003,4,producer,"Animation,Comedy,Romance"


In [3]:
analitico_titulos = """
CREATE TABLE IF NOT EXISTS analitico_titulos AS
WITH participantes AS (
    SELECT
        tconst,
        COUNT(DISTINCT nconst) as qtdParticipantes
        
    FROM title_principals
    
    GROUP BY tconst
)
SELECT 
    tb.tconst,
    tb.titleType,
    tb.originalTitle,
    tb.startYear,
    tb.endYear,
    tb.genres,
    tr.averageRating,
    tr.numVotes,
    p.qtdParticipantes
    
FROM title_basics as tb

LEFT JOIN title_ratings as tr
    ON tr.tconst = tb.tconst
    
LEFT JOIN participantes as p
    ON p.tconst = tb.tconst
"""

analitico_participantes = """
CREATE TABLE IF NOT EXISTS analitico_participantes AS

SELECT
    tp.nconst,
    tp.tconst,
    tp.ordering,
    tp.category,
    tb.genres

FROM title_principals as tp

LEFT JOIN title_basics as tb
    ON tp.tconst = tb.tconst
"""

In [4]:
queries = [analitico_titulos, analitico_participantes]

for query in queries:
    
    connection = sqlite3.connect(data_base)

    connection.execute(query)

    connection.close()

    print('Tabela Criada com sucesso!')

Tabela Criada com sucesso!
Tabela Criada com sucesso!
